In [22]:
# Importación de librerías necesarias
import pandas as pd
import joblib
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

# Cargar los datos de entrenamiento y prueba
X_train = pd.read_csv(r'D:\Desktop\t1_intro_ml\riesgo_diabetes\data\processed\X_train.csv')
y_train = pd.read_csv(r'D:\Desktop\t1_intro_ml\riesgo_diabetes\data\processed\y_train.csv')
X_test = pd.read_csv(r'D:\Desktop\t1_intro_ml\riesgo_diabetes\data\processed\X_test.csv')
y_test = pd.read_csv(r'D:\Desktop\t1_intro_ml\riesgo_diabetes\data\processed\y_test.csv')

# Cargar las características seleccionadas desde el archivo CSV
selected_features = pd.read_csv(r'D:\Desktop\t1_intro_ml\riesgo_diabetes\data\processed\selected_features.csv').squeeze().tolist()

# Cargar el modelo entrenado desde el archivo .joblib
model = joblib.load(r'D:\Desktop\t1_intro_ml\riesgo_diabetes\models\best_random_forest_model.joblib')

# Definir las variables categóricas
CATEGORICAL_VARS = ['Gender', 'Polyuria', 'Polydipsia', 'sudden weight loss', 
                    'Genital thrush', 'Itching', 'Irritability', 'delayed healing', 
                    'partial paresis', 'Alopecia']

# Utilizar LabelEncoder para codificar las variables categóricas
le = LabelEncoder()

for var in CATEGORICAL_VARS:
    if var in X_train.columns:
        X_train[var] = le.fit_transform(X_train[var].astype(str))
        X_test[var] = le.transform(X_test[var].astype(str))
    else:
        print(f"Advertencia: la variable {var} no está presente en los datos.")

# Filtrar solo las características seleccionadas que estén presentes en los datos
selected_features_present = [feat for feat in selected_features if feat in X_train.columns]

# Filtrar las características seleccionadas que están presentes
X_train_selected = X_train[selected_features_present]
X_test_selected = X_test[selected_features_present]

# Definir el pipeline de preprocesamiento sin DropFeatures
pipeline = Pipeline([

    # ===== ESCALADO =====
    # Escalar la variable Age
    ('scaler', MinMaxScaler())
])

# Ajustar el pipeline en los datos de entrenamiento (sin DropFeatures)
pipeline.fit(X_train_selected)

# Transformar los datos de prueba
X_test_transformed = pipeline.transform(X_test_selected)

# Realizar las predicciones con el modelo cargado
predictions = model.predict(X_test_transformed)

# Mostrar las predicciones
print("Predicciones:", predictions)

# Evaluar las predicciones
accuracy = accuracy_score(y_test, predictions)
print(f"Precisión del modelo: {accuracy}")

# Reporte de clasificación
report = classification_report(y_test, predictions)
print(f"Reporte de clasificación:\n{report}")


Predicciones: [0 1 1 1 0 1 0 1 1 1 1 0 1 0 0 1 1 0 0 0 0 1 0 1 1 1 0 1 1 1 1 0 1 1 0 1 1
 0 1 1 1 0 0 1 1 1 1 1 0 0 0 0]
Precisión del modelo: 0.9807692307692307
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.95      1.00      0.98        20
           1       1.00      0.97      0.98        32

    accuracy                           0.98        52
   macro avg       0.98      0.98      0.98        52
weighted avg       0.98      0.98      0.98        52



c:\Users\Juan Guzman\anaconda3\envs\datapath_ml\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
